# From JSON training events descriptions to Bioschemas markup
https://bioschemas.org/profiles/CourseInstance/0.8-DRAFT-2020_10_06/

In [68]:
import requests
import json

from rdflib import ConjunctiveGraph, URIRef, Namespace, Literal
from rdflib.namespace import RDF, RDFS, FOAF, XSD


In [74]:
events_api = 'https://catalogue.france-bioinformatique.fr/api/event'

# RDFlib graph object
G = ConjunctiveGraph()
SCHEMA = Namespace("https://schema.org/")

try:
    # first HTTP GET request to the catalogue API
    res = requests.get(events_api).json()

    count = 0
    has_next = True
    
    # we iterate over each page of results
    while has_next:
        # we iterate over each result in the results set
        for item in res['results']:

            if item.get("id") and item.get("type") and (item["type"] == "Formation"): 
                training_uri = URIRef("https://catalogue.france-bioinformatique.fr/api/trainingEvent/"+str(item['id']))
                
                #<https://schema.org/CourseInstance>
                G.add((training_uri, RDF.type, SCHEMA.CourseInstance))
                
                #print("((((((--------))))))")
                if item.get("name"):
                    #print(item["name"])
                    G.add((training_uri, SCHEMA.name, Literal(item["name"], datatype=XSD.string)))
                
                if item.get("city"):
                    G.add((training_uri, SCHEMA.location, Literal(item["city"], datatype=XSD.string)))
                
                if item.get("homepage"):
                    #print(item["homepage"])
                    #print(json.dumps(item, indent=True))
                    G.add((training_uri, SCHEMA.url, URIRef(item["homepage"])))
                    
                if item.get("description"):
                    #print(item["description"])
                    G.add((training_uri, SCHEMA.description, Literal(item["description"], datatype=XSD.string)))
                

                # TODO check that dates are valid ISO-8601 
                if item.get("dates"):
                    for date in item["dates"]:
                        if date.get("dateStart"):
                            G.add((training_uri, SCHEMA.startDate, Literal(date["dateStart"], datatype=XSD.date)))
#                            print(f"START {date['dateStart']}")
                        if date.get("dateEnd"):
                            G.add((training_uri, SCHEMA.endDate, Literal(date["dateEnd"], datatype=XSD.date)))
#                            print(f"END {date['dateStart']}")
                
            count += 1
        has_next = (res['next'] != None)
        #
        if (has_next):
            res = requests.get(res['next']).json()
except requests.exceptions.HTTPError as err:
    print(err)
    
print(f'{len(G)} triples in our RDF graph')

1240 triples in our RDF graph


In [75]:
print(G.serialize(format="turtle").decode())    

@prefix ns1: <https://schema.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://catalogue.france-bioinformatique.fr/api/trainingEvent/2059> a ns1:CourseInstance ;
    ns1:description "Principales analyses statistiques sous R, graphiques, librairies de biologie..."^^xsd:string ;
    ns1:endDate "2016-10-14"^^xsd:date ;
    ns1:location "PRABI-Doua, Villeurbanne"^^xsd:string ;
    ns1:name "Formation à R"^^xsd:string ;
    ns1:startDate "2016-10-12"^^xsd:date ;
    ns1:url <http://www.prabi.fr/spip.php> .

<https://catalogue.france-bioinformatique.fr/api/trainingEvent/2060> a ns1:CourseInstance ;
    ns1:description "Formation sur l'utilisation de base du cloud IFB"^^xsd:string ;
    ns1:location "IFB, Gif-sur-Yvette"^^xsd:string ;
    ns1:name "Formations Cloud IFB IBI-1"^^xsd:string ;
    ns1:startDa

In [76]:
print(G.serialize(format="json-ld").decode())

[
  {
    "@id": "https://catalogue.france-bioinformatique.fr/api/trainingEvent/2126",
    "@type": [
      "https://schema.org/CourseInstance"
    ],
    "https://schema.org/description": [
      {
        "@value": "Aller plus loin avec Perl afin d’être autonome pour des manipulations complexes visant à extraire et reformater des données iss..."
      }
    ],
    "https://schema.org/location": [
      {
        "@value": "Jouy en Josas"
      }
    ],
    "https://schema.org/name": [
      {
        "@value": "Cours MIGALE - Perl avancé"
      }
    ],
    "https://schema.org/startDate": [
      {
        "@type": "http://www.w3.org/2001/XMLSchema#date",
        "@value": "2017-05-11"
      }
    ],
    "https://schema.org/url": [
      {
        "@id": "http://migale.jouy.inra.fr/"
      }
    ]
  },
  {
    "@id": "https://catalogue.france-bioinformatique.fr/api/trainingEvent/2214",
    "@type": [
      "https://schema.org/CourseInstance"
    ],
    "https://schema.org/name": [
  

# How to validate our Bioschemas markup

- JSON-LD playground : https://json-ld.org/playground/
- Google structured data test tools: https://search.google.com/structured-data/testing-tool
- Google rich results tool : https://search.google.com/test/rich-results